In [1]:
#ORM
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Set up the database engine (SQLite in-memory database in this example)
engine = create_engine('sqlite:///:memory:', echo=True)

# Create a base class for the models
Base = declarative_base()

# Define a model (table) class
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

    def __repr__(self):
        return f"<User(name={self.name}, age={self.age})>"

# Create the table in the database
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Create and add new users to the session
new_user = User(name='Alice', age=30)
session.add(new_user)
session.commit()

# Query the database
for user in session.query(User).all():
    print(user)


2024-10-29 13:28:13,327 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-29 13:28:13,328 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-10-29 13:28:13,328 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-29 13:28:13,329 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2024-10-29 13:28:13,329 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-29 13:28:13,330 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	age INTEGER, 
	PRIMARY KEY (id)
)


2024-10-29 13:28:13,330 INFO sqlalchemy.engine.Engine [no key 0.00023s] ()
2024-10-29 13:28:13,331 INFO sqlalchemy.engine.Engine COMMIT
2024-10-29 13:28:13,332 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-29 13:28:13,333 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?)
2024-10-29 13:28:13,333 INFO sqlalchemy.engine.Engine [generated in 0.00030s] ('Alice', 30)
2024-10-29 13:28:13,334 INFO sqlalchemy.engine.Engine COMMIT
2024-10-29 13

C:\Users\HangDTH\AppData\Local\Temp\ipykernel_9952\4173630923.py:10: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
#SCHEMA - CENTRIC MODEL
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()
users = Table('users', metadata,
              Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('fullname', String))

actors = Table('actors', metadata,
               Column('id', Integer, primary_key=True),
               Column('name', String),
               Column('fullname', String),
               Column('body_count', Integer))

roles = Table('roles', metadata,
              Column('id', Integer, primary_key=True),
              Column('actor_id', None, ForeignKey('actors.id')),
              Column('character_name', String, nullable=False))

metadata.create_all(engine)

conn = engine.connect()
ins = actors.insert().values(name = 'Graham', fullname = 'Graham Chapman', body_count=10)
result = conn.execute(ins)
result.inserted_primary_key

print(str(ins))
ins.compile().params

INSERT INTO actors (name, fullname, body_count) VALUES (:name, :fullname, :body_count)


{'name': 'Graham', 'fullname': 'Graham Chapman', 'body_count': 10}

In [9]:
results = conn.execute(roles.insert(), [
    {'actor_id': 1, 'character_name': 'King Author'},
    {'actor_id': 1, 'character_name': 'Voice of God'},
    {'actor_id': 2, 'character_name': 'Sir Lancelot'},
    {'actor_id': 2, 'character_name': 'Black Knight'},
    {'actor_id': 3, 'character_name': 'Patsy'},
    {'actor_id': 3, 'character_name': 'Sir Bors'},
])

results.rowcount

6

In [16]:
stmt = actors.update().where(actors.c.name == 'Gram').values(name = 'Graham')
result = conn.execute(stmt)
result.rowcount

0

In [17]:
result = conn.execute(actors.delete().where(actors.c.name == 'Terry'))
result.rowcount

0

In [20]:
result = conn.execute(actors.select())
for row in result:
    print(row)

(1, 'Graham', 'Graham Chapman', 10)


In [23]:
results = conn.execute(actors.insert(), [
    {'name':'John', 'fullname':'John Cleese', 'body_count':10},
    {'name':'Terry', 'fullname':'Terry Gilliam', 'body_count':10}
])

results.inserted_primary_key_rows
results.rowcount

2

In [30]:
from sqlalchemy import select
s = select(actors.c.id, actors.c.name, actors.c.fullname)
result = conn.execute(s)
for row in result:
    print(row)

(1, 'Graham', 'Graham Chapman')
(2, 'John', 'John Cleese')
(3, 'Terry', 'Terry Gilliam')
(4, 'John', 'John Cleese')
(5, 'Terry', 'Terry Gilliam')


In [33]:
st = select(actors.c.name).order_by(actors.c.name.desc())
conn.execute(st).fetchall()

[('Terry',), ('Terry',), ('John',), ('John',), ('Graham',)]

In [36]:
st = select(actors.c.name, actors.c.fullname).limit(1).offset(1)
conn.execute(st).first()

('John', 'John Cleese')

In [41]:
from sqlalchemy.sql import func
st = select(func.count()).select_from(actors)
conn.execute(st).scalar()

5

In [42]:
s = select(actors, roles).where(actors.c.id == roles.c.actor_id)

for row in conn.execute(s):
    print(row)

(1, 'Graham', 'Graham Chapman', 10, 1, 1, 'King Author')
(1, 'Graham', 'Graham Chapman', 10, 2, 1, 'Voice of God')
(2, 'John', 'John Cleese', 10, 3, 2, 'Sir Lancelot')
(2, 'John', 'John Cleese', 10, 4, 2, 'Black Knight')
(3, 'Terry', 'Terry Gilliam', 10, 5, 3, 'Patsy')
(3, 'Terry', 'Terry Gilliam', 10, 6, 3, 'Sir Bors')


In [43]:
st = select(actors.c.name, func.count()). select_from(actors.join(roles)).group_by(actors.c.name)
conn.execute(st).fetchall()

[('Graham', 2), ('John', 2), ('Terry', 2)]

In [45]:
from sqlalchemy.sql import and_, or_, not_

st = select(actors.c.name, 
            roles.c.character_name).where(
                and_(actors.c.name.like('Gra%'), 
                    roles.c.character_name.like('Vo%'), 
                    actors.c.id == roles.c.actor_id))

conn.execute(st).fetchall()

[('Graham', 'Voice of God')]